In [ ]:
!pip install langchain-community
!pip install langchain-openai
!pip install pymupdf

In [3]:
import os
from langchain_community.document_loaders import PyMuPDFLoader


os.environ['LANGCHAIN_API_KEY'] = " "
os.environ['LANGCHAIN_PROJECT'] = "서울시 정책 전문 챗봇 BASIC.VER"
os.environ['OPENAI_API_KEY'] = " "

In [7]:
loader = PyMuPDFLoader('/home/chl/MyProject/seoul_chat/seoul.pdf')

docs = loader.load()
# 문서 요약 정보 출력
print(f"로드된 문서 수: {len(docs)}")
if docs:
    print("\n첫번째 문서 요약:")
    print(f"페이지 수 : {docs[0].metadata.get('page','N/A')}")
    print(f"페이지 수 : {docs[0].metadata.get('total_page','N/A')}")
    print(f"페이지 수 : {docs[0].metadata.get('page','N/A')}")
    print(f'내용 미리보기 : {docs[0].page_content[:100]}...')

로드된 문서 수: 158

첫번째 문서 요약:
페이지 수 : 0
페이지 수 : N/A
페이지 수 : 0
내용 미리보기 : 2024
서울시 정책 
미리 알고 알차게 즐겨요!
발 간 등 록 번 호
51-6110000-001430-10
달라지는
...


In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=100)
split = text_splitter.split_documents(docs)
print(len(split))

166


In [10]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vector_store = FAISS.from_documents(documents=split, embedding=OpenAIEmbeddings())

print(f'벡터스토어 유형 : {type(vector_store)}')
print(f'저장된 문서 수 : {vector_store.index.ntotal}')
print(f'벡터의 차원 : {vector_store.index.d}')

벡터스토어 유형 : <class 'langchain_community.vectorstores.faiss.FAISS'>
저장된 문서 수 : 166
벡터의 차원 : 1536


In [11]:
retriver = vector_store.as_retriever()

print(f'Retriver 유형 : {type(retriver)}')
print(f'검색 유형 : {retriver.search_type}')
print(f'검색 파라미터: {retriver.search_kwargs}')

Retriver 유형 : <class 'langchain_core.vectorstores.base.VectorStoreRetriever'>
검색 유형 : similarity
검색 파라미터: {}


In [13]:
from langchain_core.prompts import PromptTemplate

text = """
당신은 서울시의 정책에 대해 상세히 알고 있는 친절한 시민 상담사입니다. 당신의 임무는 서울 시민들에게 서울시의 다양한 정책을 쉽고 친절하게 설명하는 것입니다.

주어진 문맥(context)을 바탕으로 질문(question)에 답해주세요. 답변할 때는 다음 지침을 따라주세요:
1. 항상 친절하고 이해하기 쉬운 말로 설명해주세요.
2. 가능한 한 구체적인 정보를 제공하되, 불필요한 세부사항은 생략하세요.
3. 정책의 목적과 시민들에게 어떤 혜택이 있는지 강조해주세요.
4. 만약 주어진 문맥에서 답을 찾을 수 없다면, "죄송합니다. 현재 가지고 있는 정보로는 이 질문에 답하기 어렵습니다. 서울시 공식 홈페이지나 다산콜센터(120)에서 더 자세한 정보를 얻으실 수 있습니다."라고 답하세요.
5. 기술적인 용어나 정책 이름은 그대로 사용하되, 필요하다면 간단한 설명을 덧붙여주세요.

질문에 대해 단계적으로 생각하고, 논리적으로 답변을 구성해주세요.

#질문: 
{question} 

#제공된 정보: 
{context} 

#답변:
"""

prompt = PromptTemplate.from_template(text)


In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model_name = 'gpt-4o', temperature = 0)


rag_chain = (
    {"context":retriver, "question":RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)

print(f'RAG 체인 타임: {type(rag_chain)}')

RAG 체인 타임: <class 'langchain_core.runnables.base.RunnableSequence'>


In [15]:
test_q = "서울시의 주요 복지 정책은 무엇인가요?"
response = rag_chain.invoke(test_q)

print(f'질문 : {test_q}')
print(f'응답 : {response}')

질문 : 서울시의 주요 복지 정책은 무엇인가요?
응답 : 서울시의 주요 복지 정책 중 하나는 "서울시 마약관리센터"의 운영입니다. 이 센터는 서울시민 누구나 이용할 수 있으며, 마약류 중독자에게 전문의 상담과 진료를 통해 치료 보호비를 지원합니다. 2024년 10월부터 시립은평병원 내에서 운영될 예정이며, 그 전까지는 은평병원의 외래 예약 시스템을 통해 서비스를 이용할 수 있습니다.

이 정책의 주요 목적은 최근 급증하는 마약류 피해에 선제적으로 대응하고, 검사, 치료, 재활 등 통합 서비스를 제공하여 마약류 중독에 노출된 시민들을 보호하는 것입니다. 또한, 민간 중심으로 운영되던 마약류 중독 치료 서비스를 민·관이 협력하여 보다 촘촘하게 지원함으로써 중독자 회복을 돕고자 합니다.

이 외에도 서울시는 다양한 복지 정책을 통해 시민들의 삶의 질을 향상시키고자 노력하고 있습니다. 더 자세한 정보는 서울시 공식 홈페이지나 다산콜센터(120)를 통해 확인하실 수 있습니다.
